In [25]:
import pandas as pd
import glob
import json

df = pd.read_csv(
    "../parking_data_sum/parking_spot_name_data_renamed.csv",
    # dtype={"user_ID": "Int32"},
)

df = df.dropna().reset_index(drop=True)

df.insert(0, "spot_id", df.index)
df.insert(1, "spot_name", df["spot"])
df["spot_types"] = df["spot_types"].map(lambda x: json.dumps(eval(x)))
del df["created_at"]
# del df["spot_types"]
del df["spot"]
del df["index"]

df.to_csv("./spot.csv", index=False)

df


,spot_id,spot_name,lat,lon,count,spot_types,is_parking
0,0,奈良先端科学技術大学院大学 先端科学技術研究科 情報科学領域,34.731630,135.734690,14677,"[""university"", ""point_of_interest"", ""establish...",True
1,1,学際融合領域研究棟２号館,34.732260,135.732430,5131,"[""university"", ""point_of_interest"", ""establish...",True
2,2,物質創成科学領域E棟,34.730270,135.733930,4957,"[""university"", ""point_of_interest"", ""establish...",True
3,3,学研奈良登美ヶ丘駅,34.726800,135.754380,3829,"[""spa"", ""point_of_interest"", ""establishment""]",True
4,4,学際融合領域研究棟1号館,34.730930,135.735370,2679,"[""university"", ""point_of_interest"", ""establish...",True
...,...,...,...,...,...,...,...
1307,1307,御幸荘花結び,34.796163,135.252296,1,"[""lodging"", ""restaurant"", ""point_of_interest"",...",False
1308,1308,美容室リュバンドール(RUBAN D'OR),34.727334,135.715298,1,"[""hair_care"", ""beauty_salon"", ""point_of_intere...",False
1309,1309,こぐま薬局西登美ヶ丘店,34.716811,135.745120,1,"[""pharmacy"", ""health"", ""store"", ""point_of_inte...",False
1310,1310,スーパーヤオヒコ 北大和店,34.726044,135.728961,1,"[""supermarket"", ""grocery_or_supermarket"", ""sto...",False


In [26]:
import pandas as pd

from geopy.distance import geodesic

spot_df = pd.read_csv("./spot.csv")


def distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).m


def find_spot(name, lat, lon):
    target_spots = spot_df[
        spot_df.apply(lambda spot_row: spot_row["spot_name"] == name, axis=1)
    ]
    target_spots["distance"] = target_spots.apply(
        lambda x: distance(x["lat"], x["lon"], lat, lon), axis=1
    )
    target_spots = target_spots.sort_values("distance")

    if 50 < target_spots.iloc[0]["distance"]:
        print(target_spots.iloc[0])

    return target_spots.iloc[0]["spot_id"]


print(find_spot("イオンモール高の原", 34.725966, 135.791240))

result_df = pd.DataFrame()

date_list = [
    "202204",
    "202205",
    "202206",
    "202207",
    "202208",
    "202209",
    "202210",
    "202211",
    "202212",
    "202301",
    "202302",
    "202303",
]

for date in date_list:
    df = pd.read_csv(
        f"../parking_data_time_user/parking_data_time_user_{date}.csv",
        dtype={"user_ID": "Int32"},
    )
    result_df = pd.concat([result_df, df], axis=0).dropna()

result_df = result_df.reset_index()

result_df["user_id"] = result_df["user_ID"]
result_df["spot_id"] = result_df.apply(
    lambda x: find_spot(x["spot"], x["lat"], x["lon"]),
    axis=1,
)

result_df["history_id"] = result_df["history_index"]

del result_df["history_index"]
del result_df["index"]
del result_df["spot"]
del result_df["user_ID"]

result_df.to_csv("./trip.csv", index=False)

result_df


/tmp/ipykernel_40773/477371985.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_spots["distance"] = target_spots.apply(


208


,created_at,lat,lon,car,user_id,spot_id,history_id
0,2022-04-01 09:42:00+09:00,34.731556,135.734905,iMiev02,67,0,202204_0
1,2022-04-01 10:42:00+09:00,34.731556,135.734905,iMiev02,67,0,202204_0
2,2022-04-01 11:30:00+09:00,34.731605,135.734936,iMiev02,67,0,202204_0
3,2022-04-01 12:02:00+09:00,34.725523,135.789171,iMiev02,67,37,202204_0
4,2022-04-01 13:41:00+09:00,34.726816,135.754365,iMiev02,67,3,202204_2
...,...,...,...,...,...,...,...
10301,2023-03-31 19:48:00+09:00,34.726518,135.715110,LEAF (STAFF ONLY),9,68,202303_249
10302,2023-03-31 20:09:00+09:00,34.726648,135.754558,iMiev01,149,3,202303_245
10303,2023-03-31 20:16:00+09:00,34.731648,135.734976,iMiev02,156,0,202303_250
10304,2023-03-31 21:17:00+09:00,34.731648,135.734976,iMiev02,156,0,202303_250


In [13]:
import pandas as pd
import glob

result_df = pd.DataFrame()

date_list = [
    "202204",
    "202205",
    "202206",
    "202207",
    "202208",
    "202209",
    "202210",
    "202211",
    "202212",
    "202301",
    "202302",
    "202303",
]

for date in date_list:
    path = glob.glob(f"../data/{date}/history*.csv")[0]
    df = pd.read_csv(
        path,
        # dtype={"user_ID": "Int32"},
    )
    df["history_index"] = df.index.map(lambda i: f"{date}_{i}")
    result_df = pd.concat([result_df, df], axis=0).dropna()

result_df = result_df.reset_index()

result_df.insert(0, "history_id", result_df["history_index"])
result_df["distance"] = result_df["distance(km)"]
result_df["user_id"] = result_df["wallet_address"]
del result_df["distance(km)"]
del result_df["history_index"]
del result_df["wallet_address"]
del result_df["index"]

result_df.to_csv("./drive.csv", index=False)

result_df


,history_id,started_at,ended_at,from_parking_lot,to_parking_lot,car,passengers_count,distance,user_id
0,202204_0,2022-04-01 09:02,2022-04-01 12:37,NAIST,STATION,iMiev02,3,12.155552539654046,67
1,202204_1,2022-04-01 11:55,2022-04-01 13:11,NAIST,NAIST,LEAF (STAFF ONLY),1,14.526907989995856,2
2,202204_2,2022-04-01 12:41,2022-04-01 18:36,STATION,NAIST,iMiev02,3,22.336619868894612,67
3,202204_3,2022-04-01 17:44,2022-04-01 18:00,NAIST,NAIST,LEAF (STAFF ONLY),4,2.6650702613564605,8
4,202204_4,2022-04-01 18:43,2022-04-01 20:25,NAIST,NAIST,iMiev02,2,10,78
...,...,...,...,...,...,...,...,...,...
2423,202303_247,2023-03-31 17:22,2023-03-31 18:23,NAIST,NAIST,LEAF (STAFF ONLY),1,8.268403888105079,118
2424,202303_248,2023-03-31 17:44,2023-03-31 18:25,NAIST,STATION,BMW (STAFF ONLY),1,2.7994774190750578,137
2425,202303_249,2023-03-31 18:51,2023-03-31 20:43,NAIST,NAIST,LEAF (STAFF ONLY),1,5.700650492866569,9
2426,202303_250,2023-03-31 19:22,2023-04-01 05:14,NAIST,NAIST,iMiev02,1,6.73965243379831,156


In [4]:
import pandas as pd
import glob

df = pd.read_csv(
    "../data/mapping202309.csv",
    # dtype={"user_ID": "Int32"},
)

df["user_id"] = df["ID"]
df["user_type"] = df["Type"]
del df["Type"]
del df["ID"]
del df["Register"]
del df["Unregister"]

df.to_csv("./user.csv", index=False)

df


,user_id,user_type
0,1,staff
1,2,staff
2,3,staff
3,4,staff
4,5,staff
...,...,...
190,191,student
191,192,student
192,193,student
193,194,student
